In [85]:
import pandas as pd 
import glob 
import multiprocessing 
import gensim.models.word2vec as w2v
import sklearn 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import re 
import nltk 
import os 
import codecs
import glob
import logging
import multiprocessing
import pprint
from nltk.tokenize import sent_tokenize
import string


In [86]:
data = pd.read_csv('allData.csv')
data.dropna()
data = data['article']       
data = pd.DataFrame(data)
data_nan=data.dropna() 


In [87]:
def clean_text(text):

    # split into words
    from nltk.tokenize import word_tokenize
    tokens = word_tokenize(text)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word
    import string
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('french'))
    words = [w for w in words if not w in stop_words]
    return words

In [88]:
from nltk.tokenize import sent_tokenize
#sentence where each word is tokenized
sentences = []
for raw in data_nan['article']:
    if len(raw) > 0:
        sentences.append(clean_text(raw))
        


In [93]:
#step 3 - build model
#3 main tasks that vectors help with
#DISTANCE, SIMILARITY, RANKING

# Dimensionality of the resulting word vectors.
#more dimensions, more computationally expensive to train
#but also more accurate
#more dimensions = more generalized
num_features = 300
# Minimum word count threshold.
min_word_count = 3

# Number of threads to run in parallel.
#more workers, faster we train
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7

# Downsample setting for frequent words.
#0 - 1e-5 is good for this
downsampling = 1e-3

# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

In [94]:
model = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

In [95]:
model.build_vocab(sentences)

In [96]:
print("Word2Vec new vocabulary length:", len(model.wv.vocab))


Word2Vec new vocabulary length: 12087


In [97]:
from gensim.models import word2vec
print(word2vec.FAST_VERSION) 
model.train(sentences,total_examples=model.corpus_count,epochs=model.epochs)


1


(1244021, 1300170)

In [98]:
import sklearn
from sklearn import manifold
tsne = sklearn.manifold.TSNE(n_components=2, random_state=0)
all_word_vectors_matrix = model.wv.syn0

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  after removing the cwd from sys.path.


In [99]:
all_word_vectors_matrix_2d = tsne.fit_transform(all_word_vectors_matrix)
points = pd.DataFrame(
    [
        (word, coords[0], coords[1])
        for word, coords in [
            (word, all_word_vectors_matrix_2d[model.wv.vocab[word].index])
            for word in model.wv.vocab
        ]
    ],
    columns=["word", "x", "y"]
)

In [100]:
def sent_vectorizer(sent, model):
    sent_vec =[]
    numw = 0
    for w in sent:
        try:
            if numw == 0:
                sent_vec = model[w]
            else:
                sent_vec = np.add(sent_vec, model[w])
            numw+=1
        except:
            pass
     
    return np.asarray(sent_vec) / numw

In [101]:
X=[]
for sentence in sentences:
    X.append(sent_vectorizer(sentence, model))   
 
print ("========================")
print (X)
 
 
  
 
# note with some version you would need use this (without wv) 
#  model[model.vocab] 
print (model[model.wv.vocab])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [102]:
points


,word,x,y
0,mosaïque,-60.714088,-6.942445
1,fm,-60.645309,-6.906541
2,a,-55.456776,-4.086366
3,appris,-60.490646,-38.205338
4,président,-69.105614,16.915091
5,parlement,-52.866299,-33.791920
6,mohamed,-74.951790,24.393030
7,naceur,-72.548393,17.041557
8,interrompu,-61.204937,-40.341034
9,congé,-73.827156,14.968058
